In [18]:
import pandas
import os
import numpy
import plotly.express
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef, average_precision_score
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import xgboost
from sklearn.neighbors import KNeighborsClassifier


In [19]:
df = pandas.read_csv("data/Network datatset/csv/attack_4.csv", encoding="us-ascii", delimiter=',', skipinitialspace=True)
DOWN=1 #downsampling

In [20]:
print(df)

                               Time              mac_s              mac_d  \
0        2022-02-21 14:45:25.454111  74:46:a0:bd:a7:1b  e6:3f:ac:c9:a8:8c   
1        2022-02-21 14:45:25.454114  74:46:a0:bd:a7:1b  fa:00:bc:90:d7:fa   
2        2022-02-21 14:45:25.454142  74:46:a0:bd:a7:1b  0a:fe:ec:47:74:fb   
3        2022-02-21 14:45:25.454260  00:80:f4:03:fb:12  74:46:a0:bd:a7:1b   
4        2022-02-21 14:45:25.454365  e6:3f:ac:c9:a8:8c  74:46:a0:bd:a7:1b   
...                             ...                ...                ...   
5522485  2022-02-21 15:13:55.059500  00:0c:29:47:8c:0e  00:80:f4:03:fb:12   
5522486  2022-02-21 15:13:55.063690  00:0c:29:47:8c:0e  00:80:f4:03:fb:12   
5522487  2022-02-21 15:13:55.067166  00:0c:29:47:8c:0e  00:80:f4:03:fb:12   
5522488  2022-02-21 15:13:55.068124  74:46:a0:bd:a7:1b  00:80:f4:03:fb:12   
5522489  2022-02-21 15:13:55.070978  00:0c:29:47:8c:0e  00:80:f4:03:fb:12   

                 ip_s          ip_d    sport    dport   proto    flags  siz

In [21]:
X = df.copy()

In [22]:
# Remplacez les valeurs '[int]' par 'int' et transformez la colonne en numérique
X['modbus_response'] = X['modbus_response'].str.replace(r'\[(\d+)\]', r'\1', regex=True)
X['modbus_response'] = pandas.to_numeric(X['modbus_response'], errors='coerce')
X['modbus_response'] = X['modbus_response'].fillna(-1)
# La colonne est maintenant de type numérique avec des valeurs numériques
print(X['modbus_response'].unique())

[-1.000e+00  0.000e+00  4.700e+01 ...  2.233e+03  2.042e+03  1.865e+03]


In [23]:
X['modbus_response'] = X['modbus_response'].fillna(-1)

In [24]:
X = X.dropna()


In [25]:
nan_ratios = X.isna().mean()   
print(nan_ratios)

Time               0.0
mac_s              0.0
mac_d              0.0
ip_s               0.0
ip_d               0.0
sport              0.0
dport              0.0
proto              0.0
flags              0.0
size               0.0
modbus_fn          0.0
modbus_response    0.0
n_pkt_src          0.0
n_pkt_dst          0.0
label_n            0.0
label              0.0
dtype: float64


In [26]:
colonnes_catégorielles = X.select_dtypes(include=['category', 'object']).columns.tolist()

colonnes_numériques = X.select_dtypes(include=['number']).columns.tolist()

print("Colonnes catégorielles :")
print(colonnes_catégorielles)

print("\nColonnes numériques :")
print(colonnes_numériques)

Colonnes catégorielles :
['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'proto', 'modbus_fn', 'label']

Colonnes numériques :
['sport', 'dport', 'flags', 'size', 'modbus_response', 'n_pkt_src', 'n_pkt_dst', 'label_n']


In [27]:
print("nombre de fraude et non-fraude\n",X['label_n'].value_counts() )
prc=100*df['label_n'].value_counts()[1]/(X['label_n'].value_counts()[1]+X['label_n'].value_counts()[0])
print("pourcentage de fraude:",prc )

nombre de fraude et non-fraude
 label_n
0    2703946
1    1066652
Name: count, dtype: int64
pourcentage de fraude: 71.01295338299124


In [28]:
df_reduced = X.groupby('label_n').apply(lambda x: x.sample(frac=10000/X.shape[0]))
prc=100*X['label_n'].value_counts()[1]/(df_reduced['label_n'].value_counts()[1]+X['label_n'].value_counts()[0])
print("pourcentage de fraude:",prc )
X=df_reduced
print("nombre de fraude et non-fraude df_reduced:\n",df_reduced['label_n'].value_counts() )

if DOWN:
    df_class_0 = X[X['label_n'] == 0]
    df_class_1 = X[X['label_n'] == 1]
    min_rows = min(df_class_0.shape[0], df_class_1.shape[0])

    # Sous-échantillonnez la classe majoritaire (classe 0) pour égaler le nombre de lignes de la classe minoritaire (classe 1)
    df_class_0_sampled = df_class_0.sample(n=min_rows, random_state=42)

    # Concaténez les deux DataFrames équilibrés
    df_balanced = pandas.concat([df_class_0_sampled, df_class_1], ignore_index=True)

    # Mélangez les lignes pour mélanger les deux classes
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    # Vérifiez l'équilibre des classes dans le nouveau DataFrame
    print(df_balanced.shape)
    X=df_balanced
    
prc=100*X['label_n'].value_counts()[1]/(df_reduced['label_n'].value_counts()[1]+X['label_n'].value_counts()[0])
print("pourcentage de fraude:",prc )
X=df_reduced
print("nombre de fraude et non-fraude df_reduced:\n",df_reduced['label_n'].value_counts() )

pourcentage de fraude: 39.40674788262785
nombre de fraude et non-fraude df_reduced:
 label_n
0    7171
1    2829
Name: count, dtype: int64
(5658, 16)
pourcentage de fraude: 50.0
nombre de fraude et non-fraude df_reduced:
 label_n
0    7171
1    2829
Name: count, dtype: int64


In [29]:
# Store the attack_cat labels in a DataFrame Y as a 1-D array
Y = X['label_n'].copy()

In [30]:
print(Y)

label_n         
0        4570983    0
         1919268    0
         1711035    0
         1868650    0
         3254534    0
                   ..
1        2960867    1
         3991435    1
         4071078    1
         2095798    1
         2683079    1
Name: label_n, Length: 10000, dtype: int64


In [31]:
#on drop Label, Time et Label_n pour pouvoir faire l'apprentissage
#Label_n met aussi les erreur physique comme attaque, or ce ne sont pas des attaques. C'est pourquoi on utilise pas cette colonne et on créer nous même une colonne.
X.drop(['label', 'Time','label_n'], axis=1, inplace=True)

In [32]:
X = pandas.get_dummies(X)

In [33]:
# Extract train and test data through train_test_split on X and Y
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [34]:
# Importez les bibliothèques nécessaires
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Créez le modèle SVM
svm_model = SVC(kernel='rbf') 

# Entraînez le modèle sur l'ensemble d'entraînement
svm_model.fit(X_train, Y_train)

# Faites des prédictions sur l'ensemble de test
Y_pred = svm_model.predict(X_test)


print("Confusion matrix:", confusion_matrix(Y_test, Y_pred))
print("Precision:", precision_score(Y_test, Y_pred))
print("Recall:", recall_score(Y_test, Y_pred))
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Balanced accuracy:", balanced_accuracy_score(Y_test, Y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(Y_test, Y_pred))
print("AUPRC:", average_precision_score(Y_test, Y_pred))

fig = plotly.express.imshow(confusion_matrix(Y_test, Y_pred), title="Confusion matrix")
fig.show()

Confusion matrix: [[1788    0]
 [ 712    0]]
Precision: 0.0
Recall: 0.0
Accuracy: 0.7152
Balanced accuracy: 0.5
Matthews Correlation Coefficient: 0.0
AUPRC: 0.2848


/home/bastian/miniconda3/envs/ff/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

